In [1]:
import flywheel
import datetime

# Create client
fw = flywheel.Client()

# define containers & files
gear = fw.lookup('gears/xcpengine-fw')
project = fw.lookup('davis/presurgicalEpilepsy')
design_file = project.get_file('anat-minimal.dsn')

# set date for label
x = datetime.datetime.now()
date_str = '%s-%s-%s %s:%s' % (x.month, x.day, x.year, x.hour, x.minute)

In [2]:
# create subject list
fin = 'controls.txt'
sid_list = [line.rstrip('\n').split()[0] for line in open(fin)]


In [13]:
sid_list = ['C1','C2','C3','C5','C8','C10']

In [15]:
for sub in sid_list:
    ses = fw.lookup('davis/presurgicalEpilepsy/%s/ses-01' % (sub))
    print('Running analysis for subject %s' % (fw.get(ses.parents['subject']).label))
    
    # get T1
    t1 = None
    asl_img = None
    for acq in ses.acquisitions():
        if 'T1w' in acq.label:
            t1_container=fw.get(acq.id)
            for f in t1_container.files:
                if '.nii.gz' in f.name:
                    t1=f
        if 'M0' not in acq.label and acq.files[0].classification['Measurement'] == ['Perfusion'] :
            asl_container=fw.get(acq.id)
            for f in asl_container.files:
                if '.nii.gz' in f.name:
                    asl_img=f
                    
    # set inputs and config for gear
    if t1 is not None:
        inputs = {'designfile': design_file, 'img': t1 }
        config = {'analysis_type': 'struc'}
    
    alreadydone = False
    for analysis in ses.analyses:
        if analysis.files and 'anat-minimal' in analysis.label:
            print(analysis.label)
            alreadydone = True
            print("already done " + sub)
            
    #run
    if not alreadydone and asl_img is not None:
        print("not done yet " + sub)
        analysis_id = gear.run(analysis_label='WT_xcp struct ' + date_str, config=config, inputs=inputs, destination=ses)

Running analysis for subject C1
not done yet C1
Running analysis for subject C2
not done yet C2
Running analysis for subject C3
not done yet C3
Running analysis for subject C5
not done yet C5
Running analysis for subject C8
not done yet C8
Running analysis for subject C10
